In [1]:
import numpy as np
from json import load
import spacy
import nltk
import string
from nltk.corpus import stopwords
from string import punctuation
from spacy.matcher import PhraseMatcher
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc
import joblib
nlp = spacy.load("ru_core_news_sm")


In [2]:
with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discription_of_event = load(js)


In [3]:
kcm = {'знать': [], 'уметь': [], 'владеть': []}
for i in discription_of_event.values():
    for j in i:
        for z in i[j]:
            kcm[j].append(list(nlp(z).vector))


In [4]:
len(kcm.values()), len(kcm['знать']), len(kcm['знать'][0])


(3, 228, 96)

In [5]:
k = []
for i in kcm:
    for j in kcm[i]:
        k.append(j+[i])
# pd.DataFrame(k, columns=range(len(k[0])))
# sorted(k, key=len, reverse=False)
# k

In [6]:
len(k), len(k[0])

(707, 97)

In [7]:
data = pd.DataFrame(k, columns=range(len(k[0])))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(
    96, axis=1), data[96], test_size=0.25, random_state=42)


# Bayes

# Подбираем гиппер-параметры.

In [9]:
gaussian_naive_bayes_optimal_n_neighbors = GridSearchCV(GaussianNB(),
                                                        {'var_smoothing': np.arange(0, 1, 1e-1),
                                                         },
                                                        scoring=accuracy_score).fit(X_train, y_train)
gaussian_naive_bayes_optimal_n_neighbors.best_params_


c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\utils\_param_validation.py", line 175, in wrapper
    params = func_sig.bind(*args, **kwargs)
  File "C:\Users\Mike\AppData\Local\Programs\Python\Python39\lib\inspect.py", line 3062, in bind
    return self._bind(args, kwargs)
  File "C:\Users\Mike\AppData\Local\Programs\Python\Python39\lib\inspect.py", line 2988, in _bind
    raise TypeError(
TypeError: too many positional arguments

  warnings.warn(
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\m

{'var_smoothing': 0.0}

# Формула Байеса

In [10]:
gaussian_naive_bayes = GaussianNB(
    var_smoothing=gaussian_naive_bayes_optimal_n_neighbors.best_params_['var_smoothing'])
gaussian_naive_bayes.fit(X_train, y_train)
y_predicted_by_gaussian_naive_bayes = gaussian_naive_bayes.predict(X_test)


# Оценка качества 

In [11]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_gaussian_naive_bayes))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_gaussian_naive_bayes))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_gaussian_naive_bayes))



Accuracy
0.903954802259887
Confusion matrix
[[39  2  2]
 [ 7 55  3]
 [ 3  0 66]]
Precision, Recall, F
               precision    recall  f1-score   support

     владеть       0.80      0.91      0.85        43
       знать       0.96      0.85      0.90        65
       уметь       0.93      0.96      0.94        69

    accuracy                           0.90       177
   macro avg       0.90      0.90      0.90       177
weighted avg       0.91      0.90      0.90       177



In [12]:
# joblib.dump(gaussian_naive_bayes,
#             '../../PipeLines/Classifications/NaiveBayesKCMDiscriptor/nb_classification_discriptor_kcm.sav')
